# Installing everything needed 

I keep on having an issue with this stage, and I am worried about it for running the model etc. 

In [2]:
pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.1 MB/s eta 0:00:009 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 5.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.2
    Uninstalling tensorflow-2.16.2:
      Successfully uninstalled tensorflow-2.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import bertopic

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired  # Correct import

from umap import UMAP
from hdbscan import HDBSCAN
import pycountry

from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

import time

import re

import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

## 1. Creating CSV

Breaking on every line has had some issues so I will use breaking on every sentence instead. 

In [ ]:
root_folder_path = 'Text Analys Folder/UN_Speeches'

def parse_filename(filename):
    name_parts = filename.replace('.txt', '').split('_')

    if len(name_parts) < 3:
        return None, None, None  

    country = name_parts[0]
    session = name_parts[1]
    
    try:
        year = int(name_parts[2])
    except ValueError:
        return None, None, None  

    return country, session, year

def preprocess_text(text):
    # Temporarily replace dots in common abbreviations with placeholders
    abbreviation_patterns = [r'\bMs\.', r'\bMr\.', r'\bDr\.', r'\bPresident\.']
    for pattern in abbreviation_patterns:
        text = re.sub(pattern, lambda match: match.group(0).replace('.', '[DOT]'), text)
    return text

def postprocess_sentence(sentence):
    return sentence.replace('[DOT]', '.')

sentence_data = []

for subfolder in os.listdir(root_folder_path):
    subfolder_path = os.path.join(root_folder_path, subfolder)
    
    if os.path.isdir(subfolder_path) and "Session" in subfolder:
        for filename in os.listdir(subfolder_path):
            if not filename.endswith('.txt') or filename.startswith('.'):  
                continue  # Skip hidden/system files like .DS_Store
            
            country, session, year = parse_filename(filename)
            if country is None:  # Skip if parsing failed
                continue  
            
            file_path = os.path.join(subfolder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text_content = file.read()
            
            # Preprocess text
            text_content = preprocess_text(text_content)
            text_content = text_content.replace('\n', ' ')  # Replace line breaks with spaces
            
            # Split by full stops while ignoring placeholders for abbreviations
            sentences = re.split(r'(?<!\[DOT])\.\s+', text_content)
            
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence: 
                    sentence_data.append([country, session, year, postprocess_sentence(sentence)])

sentence_df = pd.DataFrame(sentence_data, columns=['Country', 'Session', 'Year', 'Sentence'])
sentence_df.to_csv('un_speeches_split_by_sentence.csv', index=False)

country_code_to_name = {country.alpha_3: country.name for country in pycountry.countries}
sentence_df['Country Name'] = sentence_df['Country'].map(country_code_to_name)



Checking what the df looks like - it seems to have worked well. Splitting on sentences seems to be more robust than splitting on paragraphs. 

In [ ]:
sentence_df.to_csv('testing_final/correct_sentence_df.csv',index = False)

In [ ]:
sentence_df = pd.read_csv("correct_sentence_df.csv")

## 2. 1 Training the BERT model

The main part with this is we would like to be able to keep the BERT model afterwards and save it so it can be reloaded. This is the part that I am struggling with. 

#### Training the BERT model with one-shot embedding
This means that we can train Bert with predefined topics that we are interested in - I have created a list of topics that we care about from before 

In [ ]:
zeroshot_topic_list = [
    #Miltiary technology
    "Miltiary Technology",
    "Nuclear Weapons",
    "Biological Weapons",
    "Chemical Weapons",
    #Dual Use Technology
    "Artifical Intelligence",
    "Quantum Technology",
    "Technology Supply chains",
    "Data Security",
    "Biological Engineering",
    "General Emerging Technologies",
    "Nuclear Energy",
    "Energy Security",
    #Civilian technology 
    "Climate change and renewable energy",
    "Electric cars",
    "Financial Technology",
    "Health Technology",
    "Technology in Education",
    #Capacity Building 
    "Digital Divide",
    "Technology Capacity Building",
    "Technology Transfer",
    #Grey Zone
    "Cyberattacks and offensive cyber",
    "Election Interference",
    "Cyber espionage"
]

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Extract sentences (assuming 'sentence_docs' is a list of sentences)
sentence_docs = sentence_df["Sentence"].tolist()

# Generate embeddings for the sentences
sentence_embeddings = embedding_model.encode(sentence_docs, show_progress_bar=True)


Batches:   0%|          | 0/16323 [00:00<?, ?it/s]

In [ ]:
topic_model = BERTopic(
    embedding_model="all-MiniLM-L6-v2", 
    min_topic_size=15,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.5,
    representation_model=KeyBERTInspired()
)
topics, _ = topic_model.fit_transform(sentence_docs, )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
sentence_df['Topic'] = topics

In [ ]:
# Assuming `sentence_embeddings` is a 2D numpy array with shape (374959, 384)
sentence_df['Embedding'] = [embedding for embedding in sentence_embeddings]


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.head(26)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,235193,-1_palestinian_israel_peace_nations,"[palestinian, israel, peace, nations, poverty,...",[We will do our best to make good use of this...
1,0,4270,Nuclear Weapons,"[treaty, nuclear, disarmament, proliferation, ...",[The Treaty on the Non-Proliferation of Nuclea...
2,1,80,Biological Weapons,"[biowarfare, weapons, convention, biological, ...",[The need to establish a verification system f...
3,2,480,Chemical Weapons,"[disarmament, syria, chemical, weapons, syrian...",[The Convention on chemical weapons is a signi...
4,3,25,Artifical Intelligence,"[intelligence, intelligent, ai, smarter, human...","[Tireless determination, innovation and intell..."
5,4,10,Quantum Technology,"[quantum, advances, leap, technology, developm...",[The quantum leap in transport and communicati...
6,5,41,Technology Supply chains,"[suppliers, logistics, supply, chains, industr...",[Damaged supply chains are close to full rest...
7,6,34,Data Security,"[privacy, security, data, safeguarding, secure...","[In this respect, we must reconcile the develo..."
8,7,5,Biological Engineering,"[biotechnology, cloning, genetic, synthetic, e...","[Another key issue for humanity, the future of..."
9,8,115,General Emerging Technologies,"[technologies, technology, technological, deve...","[On the one hand, new technologies are emergin..."


In [ ]:
topic_info.shape

(2027, 5)

### 2.2 Save the model

In [ ]:
topic_model.save("bertopic_model_2", save_embedding_model=True)

2025-03-10 17:23:11,833 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


### 2.3 Load the pre-saved model 

In [ ]:

topic_model = BERTopic.load("bertopic_model")


### 2.4 Save Sentence Df

In [ ]:
sentence_df.to_csv("sentence_df_complete.csv", index=False)


## 3.1 Visualisation of topics - Seeing top words in the Zeroshot topics 
###  Rename the topics so that they are represented correctly in the various visualisations 

In [ ]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,
                                                 topic_prefix=False,
                                                 word_length=10,
                                                 separator=", ")


# Now assign the generated labels to the topics
topic_model.set_topic_labels(topic_labels)


In [ ]:
# Create a mapping dictionary from the topic model
topic_mapping = {i: label for i, label in enumerate(topic_labels)}

# Apply the mapping to the dataframe
sentence_df["Topic Name"] = sentence_df["Topic"].map(topic_mapping)

# Verify changes
sentence_df.head()



,Country,Session,Year,Sentence,Country Name,Topic,Embedding,Topic Name
0,BRB,73,2018,Let me begin by congratulating Ms. María Ferna...,Barbados,77,"[-0.036270916, -0.0026696608, 0.0026795529, -0...","helmets, peacekeepi, helmet"
1,BRB,73,2018,"However, I would like to pause at this stage, ...",Barbados,-1,"[0.015536962, 0.029011553, 0.023046829, -0.011...",NaN
2,BRB,73,2018,Those events include the transit of a tropical...,Barbados,-1,"[0.067203976, 0.04081179, 0.11460312, 0.070455...",NaN
3,BRB,73,2018,Those events are of great concern because the ...,Barbados,1584,"[0.060707532, 0.074981414, 0.02815113, 0.03468...","discrimina, discrimina, discrimina"
4,BRB,73,2018,"I ask myself, what does all that matter? Last ...",Barbados,1073,"[0.09178345, 0.048956443, 0.018212734, 0.02262...","samoa, samoans, samoan"


In [ ]:
zeroshot_topic_dict = {
    0: "Nuclear Weapons",
    1: "Biological Weapons",
    2: "Chemical Weapons",
    3: "Artificial Intelligence",
    4: "Quantum Technology",
    5: "Supply Chains",
    6: "Data Security",
    7: "Biological Engineering",
    8: "General Emerging Technologies",
    9: "Nuclear Energy",
    10: "Energy Security",
    11: "Climate Change and Renewable Energy",
    12: "Electric Cars",
    13: "Financial Technology",
    14: "Health Technology",
    15: "Technology in Education",
    16: "Digital Divide",
    17: "Technology Capacity Building",
    18: "Technology Transfer",
    19: "Cyberattacks and Offensive Cyber",
    20: "Election Interference",
    21: "Cyber Espionage"
}

topic_model.set_topic_labels(zeroshot_topic_dict) 

In [ ]:
topic_model.set_topic_labels(zeroshot_topic_dict)

# Map the numeric topics to their descriptive labels in the dataframe
sentence_df["Topic Name"] = sentence_df["Topic"].map(zeroshot_topic_dict)

# Verify the changes
sentence_df.head()



,Country,Session,Year,Sentence,Country Name,Topic,Embedding,Topic Name
0,BRB,73,2018,Let me begin by congratulating Ms. María Ferna...,Barbados,77,"[-0.036270916, -0.0026696608, 0.0026795529, -0...",NaN
1,BRB,73,2018,"However, I would like to pause at this stage, ...",Barbados,-1,"[0.015536962, 0.029011553, 0.023046829, -0.011...",NaN
2,BRB,73,2018,Those events include the transit of a tropical...,Barbados,-1,"[0.067203976, 0.04081179, 0.11460312, 0.070455...",NaN
3,BRB,73,2018,Those events are of great concern because the ...,Barbados,1584,"[0.060707532, 0.074981414, 0.02815113, 0.03468...",NaN
4,BRB,73,2018,"I ask myself, what does all that matter? Last ...",Barbados,1073,"[0.09178345, 0.048956443, 0.018212734, 0.02262...",NaN


In [ ]:

zeroshot_topic_ids = topic_info[topic_info["Name"].isin(zeroshot_topic_list)]["Topic"].tolist()

## Barchart - showing if topics seem correct 

In [ ]:
topic_model.visualize_barchart(topics=zeroshot_topic_ids, custom_labels = True )


## Visualise intertopic distance for zeroshot-topics 

In [ ]:
import json

# Generate interactive visualisation
fig = topic_model.visualize_topics(topics=zeroshot_topic_ids)



In [ ]:
topic_vis = topic_model.visualize_topics(
    topics=zeroshot_topic_ids,  # Replace with your topic IDs of interest
    custom_labels=True
)

# Show the visualization
topic_vis.update_layout(
    title="Topic Visualization with Custom Labels",
    title_x=0.5,  # Center the title
    showlegend=True
)

topic_vis.show()

## Visualise connections between topics 

In [ ]:
topic_model.visualize_hierarchy(topics = zeroshot_topic_ids, custom_labels = True)

## Heatmap to visualise similarity between topics 

In [ ]:
heatmap = topic_model.visualize_heatmap(topics=zeroshot_topic_ids, custom_labels=True)

heatmap

## Visualise Network connections between countries


In [ ]:
zeroshot_df = sentence_df[sentence_df["Topic Name"].isin(zeroshot_topic_dict.values())]

zeroshot_df = zeroshot_df[zeroshot_df["Topic Name"] != "Nuclear Weapons"]
zeroshot_df = zeroshot_df[zeroshot_df["Topic Name"] != "Climate Change and Renewable Energy"]

In [ ]:
# Define the groups
groups = {
    "ASEAN": ["Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"],
    
    # European Union subgroups
    "Original_EU_plus": ["Belgium", "France", "Germany", "Italy", "Luxembourg", "Netherlands", "Austria", "Ireland"],
    "Baltic_Nordic_States": ["Estonia", "Latvia", "Lithuania", "Denmark", "Finland", "Sweden"],
    "Eastern_Europe": ["Poland", "Czech Republic", "Slovakia", "Hungary", "Romania", "Bulgaria"],
    "Southern_Europe": ["Portugal", "Spain", "Greece", "Cyprus", "Malta"],

    # African Union subgroups
    "West_Africa": ["Benin", "Burkina Faso", "Cape Verde", "Ivory Coast", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Liberia", "Mali", "Mauritania", "Niger", "Nigeria", "Senegal", "Sierra Leone", "Togo"],
    "Central_Africa": ["Cameroon", "Central African Republic", "Chad", "Congo", "Democratic Republic of the Congo", "Equatorial Guinea", "Gabon"],
    "East_Africa": ["Burundi", "Djibouti", "Eritrea", "Ethiopia", "Kenya", "Madagascar", "Malawi", "Mauritius", "Mozambique", "Rwanda", "Seychelles", "Somalia", "South Sudan", "Sudan", "Tanzania", "Uganda", "Zambia", "Zimbabwe"],
    "Southern_Africa": ["Angola", "Botswana", "Eswatini", "Lesotho", "Namibia", "South Africa"],
    
    # Middle East
    "Middle_East": ["Afghanistan", "Bahrain", "Cyprus", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman", "Palestine", "Qatar", "Saudi Arabia", "Syria", "Turkey", "United Arab Emirates", "Yemen"],

    # Latin America & Caribbean subgroups
    "Mexico_Central_America": ["Mexico", "Guatemala", "Belize", "Honduras", "El Salvador", "Nicaragua", "Costa Rica", "Panama"],
    "Southern America": ["Colombia", "Ecuador", "Peru", "Bolivia", "Venezuela", "Argentina", "Chile", "Uruguay", "Paraguay"],
    "Caribbean": ["Antigua and Barbuda", "Bahamas", "Barbados", "Cuba", "Dominica", "Dominican Republic", "Grenada", "Haiti", "Jamaica", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Suriname", "Trinidad and Tobago"],

    # Indian Subcontinent
    "Indian_Subcontinent": ["India", "Pakistan", "Bangladesh", "Nepal", "Sri Lanka", "Maldives", "Bhutan"],

    # Korea, Japan, Australia, New Zealand
    "Korea_Japan_Australia_NewZealand": ["South Korea", "North Korea", "Japan", "Australia", "New Zealand"]
}

# Always include these major countries
always_include = ["United Kingdom", "United States", "China"]

# Function to create a filtered DataFrame for each group
def filter_by_group(group_name):
    return zeroshot_df[zeroshot_df["Country Name"].isin(groups[group_name] + always_include)]

# Creating separate DataFrames for each subgroup
ASEAN_df = filter_by_group("ASEAN")
Original_EU_df = filter_by_group("Original_EU_plus")
Baltic_Nordic_States_df = filter_by_group("Baltic_Nordic_States")
Eastern_Europe_df = filter_by_group("Eastern_Europe")
Southern_Europe_df = filter_by_group("Southern_Europe")

West_Africa_df = filter_by_group("West_Africa")
Central_Africa_df = filter_by_group("Central_Africa")
East_Africa_df = filter_by_group("East_Africa")
Southern_Africa_df = filter_by_group("Southern_Africa")

Middle_East_df = filter_by_group("Middle_East")

Mexico_Central_America_df = filter_by_group("Mexico_Central_America")
Southern_America_df = filter_by_group("Southern America")
Caribbean_df = filter_by_group("Caribbean")

Indian_Subcontinent_df = filter_by_group("Indian_Subcontinent")

Korea_Japan_Australia_NewZealand_df = filter_by_group("Korea_Japan_Australia_NewZealand")



In [ ]:
import networkx as nx
import plotly.graph_objects as go
import plotly.colors as pc

# List of DataFrames to iterate through, using their group names
# List of DataFrames to iterate through, using their group names
group_dfs = {
    "ASEAN": ASEAN_df,
    "Original EU": Original_EU_df,
    "Baltic States": Baltic_Nordic_States_df,  # Corrected to match the variable name
    "Eastern Europe": Eastern_Europe_df,
    "Southern Europe": Southern_Europe_df,
    "West Africa": West_Africa_df,
    "Central Africa": Central_Africa_df,
    "East Africa": East_Africa_df,
    "Southern Africa": Southern_Africa_df,
    "Middle East": Middle_East_df,
    "Mexico Central America": Mexico_Central_America_df,
    "Southern America": Southern_America_df,  # Corrected to match the variable name
    "Caribbean": Caribbean_df,
    "Indian Subcontinent": Indian_Subcontinent_df,  # Added the Indian Subcontinent group
    "Korea Japan Australia New Zealand": Korea_Japan_Australia_NewZealand_df  # Added Korea, Japan, Australia, New Zealand
}


# Get the "Light24" color palette
light24_colors = pc.qualitative.Pastel

# Iterate through each DataFrame
for group_name, dataframe in group_dfs.items():
    print(f"Generating network graph for: {group_name}")

    # Get unique countries and topics
    unique_countries = set(dataframe["Country Name"].unique())
    unique_topics = set(dataframe["Topic Name"].unique())

    # Aggregate edge weights
    edge_weights = dataframe.groupby(["Country Name", "Topic Name"]).size().reset_index(name="Weight")

    # Filter out countries that do not have topic connections
    connected_countries = set(edge_weights["Country Name"])
    unique_countries = unique_countries.intersection(connected_countries)

    # Create Graph
    G = nx.Graph()

    # Add nodes
    G.add_nodes_from(unique_countries, bipartite=0)  # Countries
    G.add_nodes_from(unique_topics, bipartite=1)    # Topics

    # Add edges with weights
    for _, row in edge_weights.iterrows():
        G.add_edge(row["Country Name"], row["Topic Name"], weight=row["Weight"])

    # Node positioning
    pos = nx.kamada_kawai_layout(G)

    # Assign each country a unique color
    country_colors = {country: light24_colors[i % len(light24_colors)] for i, country in enumerate(unique_countries)}

    # Create edge traces
    edge_traces = []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        weight = edge[2]["weight"]
        line_width = max(1, weight * 0.5)  

        edge_color = country_colors.get(edge[0], "darkgrey")

        edge_trace = go.Scatter(
            x=[x0, x1, None], y=[y0, y1, None],
            line=dict(width=line_width, color=edge_color),
            hoverinfo='text',
            text=[f"{edge[0]} ↔ {edge[1]}: {weight} mentions"],
            mode='lines'
        )
        edge_traces.append(edge_trace)

    # Create node traces
    node_x, node_y, node_text, node_color = [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_color.append(country_colors.get(node, "darkgrey"))

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        marker=dict(size=20, color=node_color, line=dict(width=2, color='black')),
        text=node_text,
        textposition="top center",
        hoverinfo="text"
    )

    # Create the figure
    fig = go.Figure(data=edge_traces + [node_trace])
    fig.update_layout(
        title=f"{group_name} Countries & Technology Mentions Network",
        showlegend=False,
        hovermode="closest",
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white"
    )

    fig.show()


Generating network graph for: ASEAN


Generating network graph for: Original EU


Generating network graph for: Baltic States


Generating network graph for: Eastern Europe


Generating network graph for: Southern Europe


...

Generating network graph for: Mexico Central America


Generating network graph for: Southern America


Generating network graph for: Caribbean


Generating network graph for: Indian Subcontinent


Generating network graph for: Korea Japan Australia New Zealand


## Visualise the topics over time

In [ ]:
import plotly.express as px

# Filter for only zero-shot topics (without excluding any)
filtered_df = sentence_df[sentence_df["Topic Name"].isin(zeroshot_topic_dict.values())]

# Aggregate mentions per topic per year
topic_trends = filtered_df.groupby(["Year", "Topic Name"]).size().reset_index(name="Mentions")

# Get unique topics to create a colour mapping
topics = topic_trends["Topic Name"].unique()
pastel_palette = px.colors.qualitative.Pastel
color_map = {topic: pastel_palette[i % len(pastel_palette)] for i, topic in enumerate(topics)}

# Plot with Plotly
fig = px.line(
    topic_trends,
    x="Year",
    y="Mentions",
    color="Topic Name",  # Each line is a topic
    markers=True,
    title="Overall Trends in Technology Topics Over Time",
    labels={"Mentions": "Number of Mentions"},
    template="plotly_white"
)

# Extend the x-axis range and increase tick frequency
fig.update_layout(
    xaxis=dict(
        tickmode="linear",  # Ensure evenly spaced years
        dtick=2  # Show every year on the x-axis
    ),
    showlegend=True
)

# Apply pastel colours manually
for trace in fig.data:
    trace.line.color = color_map[trace.name]  # Match topic name to pastel colour

fig.show()

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10, custom_labels=True)

# Creating Groups for aggregation 

In [ ]:
sentence_df.head()

,Country,Session,Year,Sentence,Country Name,Embedding
0,BRB,73,2018,Let me begin by congratulating Ms. María Ferna...,Barbados,"[-0.036270916, -0.0026696608, 0.0026795529, -0..."
1,BRB,73,2018,"However, I would like to pause at this stage, ...",Barbados,"[0.015536962, 0.029011553, 0.023046829, -0.011..."
2,BRB,73,2018,Those events include the transit of a tropical...,Barbados,"[0.067203976, 0.04081179, 0.11460312, 0.070455..."
3,BRB,73,2018,Those events are of great concern because the ...,Barbados,"[0.060707532, 0.074981414, 0.02815113, 0.03468..."
4,BRB,73,2018,"I ask myself, what does all that matter? Last ...",Barbados,"[0.09178345, 0.048956443, 0.018212734, 0.02262..."


In [ ]:
# Define topic-to-group mapping by topic numbers
topic_groups = {
    "Military Technology": [0, 1, 2],  # Nuclear Weapons, Biological Weapons, Chemical Weapons
    "Dual Use Technology": [3, 4, 5, 6, 7, 8, 9, 10, 19, 20, 21, 12, 13, 14, 15, 16, 17, 18],  # AI, Quantum Tech, etc.
    "Civilian Technology": [11, 12, 13, 14, 15, 16, 17, 18],  # Climate change, Electric cars, etc.
    "Capacity Building": [16, 17, 18],  # Digital Divide, Tech Capacity Building, etc.
    "Grey Zone": [19, 20, 21]  # Cyberattacks, Election Interference, etc.
}

# Function to assign group based on topic number(s)
def assign_group_by_number(topics):
    # If 'topics' is a single integer, convert it into a list
    if isinstance(topics, int):
        topics = [topics]  # Make it iterable

    # Check which group the topics belong to
    for group, topic_list in topic_groups.items():
        if any(topic in topic_list for topic in topics):
            return group
    return "Uncategorized"  # Default if no match found

# Apply function to create "group" column based on topic numbers
sentence_df["group"] = sentence_df["Topic"].apply(assign_group_by_number)



In [ ]:
zeroshot_topic_dict = {
    0: "Nuclear Weapons",
    1: "Biological Weapons",
    2: "Chemical Weapons",
    3: "Artificial Intelligence",
    4: "Quantum Technology",
    5: "Supply Chains",
    6: "Data Security",
    7: "Biological Engineering",
    8: "General Emerging Technologies",
    9: "Atomic Energy",
    10: "Energy Security",
    11: "Climate Change and Renewable Energy",
    12: "Electric Cars",
    13: "Financial Technology",
    14: "Health Technology",
    15: "Technology in Education",
    16: "Digital Divide",
    17: "Technology Capacity Building",
    18: "Technology Transfer",
    19: "Cyberattacks and Offensive Cyber",
    20: "Election Interference",
    21: "Cyber Espionage"

In [ ]:
# Filter DataFrames based on the group
df_military = sentence_df[sentence_df["group"] == "Military Technology"]
df_dual_use = sentence_df[sentence_df["group"] == "Dual Use Technology"]
df_civilian = sentence_df[sentence_df["group"] == "Civilian Technology"]
df_capacity_building = sentence_df[sentence_df["group"] == "Capacity Building"]
df_grey_zone = sentence_df[sentence_df["group"] == "Grey Zone"]

# Saving the DataFrames as CSV files
df_military.to_csv('df_military.csv', index=False)
df_dual_use.to_csv('df_dual_use.csv', index=False)
df_civilian.to_csv('df_civilian.csv', index=False)
df_capacity_building.to_csv('df_capacity_building.csv', index=False)
df_grey_zone.to_csv('df_grey_zone.csv', index=False)



In [ ]:
df_military

,Country,Session,Year,Sentence,Country Name,Topic,Embedding,Topic Name,group
48,BRB,73,2018,"I repeat, it led to two world wars and an atom...",Barbados,0,"[-0.01074151, 0.08483641, -0.014866305, 0.0476...",Nuclear Weapons,Military Technology
374,SWE,73,2018,We are witnessing a deeply worrisome nuclear r...,Sweden,0,"[0.029459324, 0.041300286, -0.034867812, 0.035...",Nuclear Weapons,Military Technology
376,SWE,73,2018,The framework of the Treaty on the Non-Prolife...,Sweden,0,"[0.005601457, 0.04289692, -0.0617467, -0.04264...",Nuclear Weapons,Military Technology
1050,ALB,73,2018,We support all the actions taken by the intern...,Albania,2,"[-0.011582497, 0.044264536, 0.013668734, -0.06...",Chemical Weapons,Military Technology
1210,BOL,73,2018,"The second major threat is the arms race, the ...","Bolivia, Plurinational State of",0,"[0.048457425, 0.045786474, -0.022934327, 0.061...",Nuclear Weapons,Military Technology
...,...,...,...,...,...,...,...,...,...
520349,TUR,69,2014,"If 2,000 people are killed by chemical weapons...",Türkiye,2,"[0.10385505, -0.017613554, -0.01746026, -0.005...",Chemical Weapons,Military Technology
520352,TUR,69,2014,Let it be understood that the use of any kind...,Türkiye,2,"[0.080962926, 0.07141191, -0.031963583, -0.004...",Chemical Weapons,Military Technology
521583,LUX,69,2014,The proliferation of weapons of mass destruct...,Luxembourg,0,"[-0.013056871, -0.0032054023, 0.026966093, 0.0...",Nuclear Weapons,Military Technology
521655,TKM,69,2014,We call for consistent strengthening of intern...,Turkmenistan,0,"[0.0061750985, 0.03824251, 0.01598714, -0.0742...",Nuclear Weapons,Military Technology


In [ ]:


# Ensure full text display for long sentences
pd.set_option('display.max_colwidth', None)

# Filter for USA in 2021
us_2021_sentences = df_dual_use[df_dual_use["Country"] == "USA"]


# Filter for Estonia between 2010 and 2021 (if you meant 2023, adjust the comment or range)
estonia_2010_2021_sentences = df_dual_use[(df_dual_use["Country"] == "EST") & (df_dual_use["Year"].between(2010, 2021))]

china_2010_2023_sentences = df_dual_use[(df_dual_use["Country"] == "CHN") & (df_dual_use["Year"].between(2010, 2023))]

china_2010_2023_sentences[["Year", "Sentence", "Topic Name"]].sort_values(by="Year", ascending=True)



,Year,Sentence,Topic Name
244251,2010,"It is a leading producer of many important products, but it remains at the lower end of the global industrial chain",Supply Chains
244253,2010,"In many cases, we have to rely on imports to meet the demand for core technologies",Technology Transfer
244295,2010,China has formulated medium- and long-term development programmes on education and on science and technology,Technology in Education
316483,2011,We should foster a security environment conducive to stability and development,Energy Security
383464,2012,We should seek common security amid growing interdependence,Energy Security
108195,2016,"Other issues to be faced are energy and resource security, food security and financial security",Energy Security
466797,2022,We have endeavoured to build a system of high-standard opening-up and to safeguard the security and stability of global industrial and supply chains,Supply Chains
466829,2022,"In response to the shared concerns of various countries about data security, we have launched the Global Initiative on Data Security as a reference for formulating rules on global data security",Data Security
466842,2022,"To achieve a fundamental solution, it is important to address the legitimate security concerns of all parties concerned and build a balanced, effective and sustainable security architecture",Energy Security


## Cosine Similarity - All years
Starting with miltiary technology creating cosine similarity for every value 

## First make a rolling average chart for every year

In [ ]:
# Group countries into different blocs using three-letter country codes
country_blocs = {
    "Total EU": [
        "AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", 
        "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"
    ],
    "China": ["CHN"],
    "US": ["USA"],
    "UK": ["GBR"],
    "ASEAN": [
        "BRN", "KHM", "IDN", "LAO", "MYS", "MMR", "PHL", "SGP", "THA", "VNM"
    ],
    "GRULAC": [
        "ARG", "BOL", "BRA", "CHL", "COL", "CRI", "CUB", "DOM", "ECU", "SLV", "GTM", "HND", "MEX", "NIC", "PAN", "PRY", "PER", "URY", "VEN"
    ],
    "African Union": [
        "DZA", "AGO", "BEN", "BWA", "BFA", "BDI", "CPV", "CMR", "CAF", "TCD", "COM", "COG", "COD", "DJI", "EGY", "GNQ", "ERI", "SWZ", "ETH", 
        "GAB", "GMB", "GHA", "GIN", "GNB", "CIV", "KEN", "LSO", "LBR", "LBY", "MDG", "MWI", "MLI", "MRT", "MUS", "MAR", "MOZ", "NAM", "NER", 
        "NGA", "RWA", "STP", "SEN", "SYC", "SLE", "SOM", "ZAF", "SSD", "SDN", "TGO", "TUN", "UGA", "ZMB", "ZWE"
    ],
    "Aligned Indo-Pacific": [
        "AUS", "NZL", "JPN", "KOR"
    ],
    "Non-Aligned Movement (NAM)": [
        "AFG", "BHR", "IRN", "IRQ", "JOR", "KAZ", "KWT", "KGZ", "LBN", "MDV", "MNG", "PRK", "OMN", "PSE", "QAT", "TJK", "TKM", 
        "ARE", "UZB", "YEM", "BGD", "BTN", "NPL", "LKA", "AZE", "BLR","IND"
    ]
}

def assign_bloc(country_code):
    for bloc, country_codes in country_blocs.items():
        if country_code in country_codes:
            return bloc
    return "Other"  

# Define the function to process each dataframe
def process_df(df):
    # Assign bloc based on country codes
    df['bloc'] = df['Country'].apply(assign_bloc)

    # Create a rolling year range (-2 to +2)
    df['New Year'] = df['Year']
    year_range = list(range(-2, 2))  # [-2, -1, 0, +1, +2]
    dfs = []

    for offset in year_range:
        df_copy = df.copy()
        df_copy['New Year'] = df_copy['Year'] + offset
        dfs.append(df_copy)

    # Concatenate expanded dataframes
    df_expanded = pd.concat(dfs, ignore_index=True)

    # Group by "bloc" and "New Year" to compute average embeddings
    df_grouped = df_expanded.groupby(['New Year', 'Country Name'], as_index=False)['Embedding'].apply(
        lambda x: np.mean(np.vstack(x), axis=0)
    ).reset_index()



    return df_grouped

# Apply the function to each dataframe
df_military_processed = process_df(df_military)
df_dual_use_processed = process_df(df_dual_use)
df_civilian_processed = process_df(df_civilian)
df_capacity_building_processed = process_df(df_capacity_building)
df_grey_zone_processed = process_df(df_grey_zone)


In [ ]:
df_dual_use_processed.to_csv("dual_use_processed.csv", index=False)
df_military_processed.to_csv("military_processed.csv", index=False)
df_civilian_processed.to_csv("civilian_processed.csv", index=False)

In [ ]:
df_dual_use_processed= pd.read_csv("dual_use_processed.csv")
df_military_processed = pd.read_csv("military_processed.csv")
df_civilian_processed = pd.read_csv("civilian_processed.csv")

## Plot for Dual-Use  Technoogy 

### Asean with dual use 

In [ ]:
import plotly.colors as pc
import plotly.graph_objects as go

base_number = 2021
# List of ASEAN countries to compare against
asean_countries = ["Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"]
# Main countries to compare against
main_countries = ["United States", "United Kingdom", "China", "Germany"]

def calculate_cosine_with_base_year(df, countries_of_interest, base_year=base_number, embeddings_column='Embedding'):
    similarity_matrix = []

    # Remove the years 1990, 1991, and 2024
    df = df[~df['New Year'].isin([1990, 1991, 2024, 2025])]

    # Extract the embeddings for the base year (2021) for the countries of interest
    base_year_data = df[df['New Year'] == base_year]
    base_embeddings = {
        country: base_year_data[base_year_data['Country Name'] == country][embeddings_column].values.tolist()
        for country in countries_of_interest
    }

    # Iterate over each year in the "New Year" column
    for year in df['New Year'].unique():
        year_data = df[df['New Year'] == year]
        year_matrix = {'New Year': year}

        # Get embeddings for all countries in the given year
        year_embeddings = {
            country: year_data[year_data['Country Name'] == country][embeddings_column].values.tolist()
            for country in year_data['Country Name'].unique()
        }

        # Compare the base year (2021) for each country of interest with all other countries
        for country in countries_of_interest:
            if len(base_embeddings[country]) > 0:
                for other_country, other_country_embedding in year_embeddings.items():
                    if len(other_country_embedding) > 0:
                        similarity = cosine_similarity([base_embeddings[country][0]], [other_country_embedding[0]])
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = similarity[0][0]
                    else:
                        # If no data for the other country in the given year, assign None
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = None
            else:
                # If no data for the country in the base year, assign None
                year_matrix[f'Cosine Similarity {country}'] = None

        similarity_matrix.append(year_matrix)

    similarity_df = pd.DataFrame(similarity_matrix)

    # Drop columns where more than 40% of the values are NaN
    threshold = len(similarity_df) * 0.4  # 40% of the rows
    similarity_df = similarity_df.dropna(thresh=threshold, axis=1)

    # Ensure that only columns with less than 40% NaN values are kept
    return similarity_df

# Compute the cosine similarity with the base year for ASEAN countries
cosine_sim_dual_use = calculate_cosine_with_base_year(df_dual_use_processed, asean_countries + main_countries)

df_list = [cosine_sim_dual_use]  # You can add cosine_sim_military, cosine_sim_civilian as needed
df_names = ["Dual-Use"]  # "Military", "Civilian" as needed

# Use Plotly's pastel color palette
pastel_palette = pc.qualitative.Pastel
color_map = {country: pastel_palette[i % len(pastel_palette)] for i, country in enumerate(asean_countries)}

# Loop through each DataFrame (Dual Use, Military, Civilian)
for cosine_sim_df, name in zip(df_list, df_names):
    for reference_country in main_countries:  # The main countries (e.g., United States)
        # Step 1: Filter for columns containing the relevant cosine similarity for each country
        filtered_df = cosine_sim_df.filter(like=f"Cosine Similarity {reference_country} vs", axis=1)

        # Step 2: Keep 'New Year' and rename columns to remove redundant text
        melted_df = cosine_sim_df[['New Year']].join(filtered_df)
        melted_df.columns = melted_df.columns.str.replace(f"Cosine Similarity {reference_country} vs ", "")

        # Step 3: Convert to long format for Plotly
        melted_df = melted_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')

        # Step 4: Convert NaN to None for proper Plotly rendering
        melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

        # Step 5: Calculate the average similarity for ASEAN countries for each year
        asean_df = melted_df[melted_df['Country'].isin(asean_countries)]
        avg_asean_similarity = asean_df.groupby('New Year')['Similarity'].mean().reset_index()

        # Step 6: Extract the last row for each country to add labels at the end
        last_row = melted_df.groupby('Country').tail(1)

        # Step 7: Create the line plot with Plotly's graph_objects for more flexibility
        fig = go.Figure()

        # Add the lines for each ASEAN country
        for country in melted_df['Country'].unique():
            if country in asean_countries:  # Only plot ASEAN countries
                country_data = melted_df[melted_df['Country'] == country]
                fig.add_trace(go.Scatter(
                    x=country_data['New Year'],
                    y=country_data['Similarity'],
                    mode='lines+markers',
                    name=country,
                    line=dict(shape='spline', color=color_map.get(country, 'black')),
                    connectgaps=True,  # Connect gaps where NaN values are present
                ))

        # Step 8: Add the average ASEAN similarity line
        fig.add_trace(go.Scatter(
            x=avg_asean_similarity['New Year'],
            y=avg_asean_similarity['Similarity'],
            mode='lines+markers',
            name='Average ASEAN Similarity',
            line=dict(shape='spline', dash='dot', color='black'),
            connectgaps=True  # Add the average ASEAN similarity line
        ))

        for i, d in enumerate(fig.data):
            fig.add_scatter(
            x=[d.x[-1]], 
            y=[d.y[-1]],
            mode='markers+text',
            text=[d.name],  # Display the country name
            textfont=dict(color=d.line.color, size=10),  # Set the text color to match the line color and make the font smaller
            textposition='middle right',  # Position the text at the right of the last point
            marker=dict(color=d.line.color, size=5),  # Set the marker to match the line color
            legendgroup=d.name,  # Group in legend for consistency
            showlegend=False  # Don't show in legend
            )

        # Step 9: Update the layout
        fig.update_layout(
            title=f'"Cosine Similarity Trends ({reference_country} vs {group_name}) ({name} Data)',
            xaxis_title='Year',
            yaxis_title='Cosine Similarity',
            yaxis=dict(range=[0, 1]),
            xaxis=dict(tickmode='array', tickvals=[year for year in range(1990, 2025, 2)]),  # Adjust x-axis ticks every 2 years
            showlegend=False,
            template="plotly_white",
            hovermode='closest'
        )

        # Show the plot
        fig.show()



In [ ]:
import plotly.graph_objects as go
import plotly.colors as pc
from plotly.subplots import make_subplots

base_number = 2021

# Define groups for comparison
groups = {
    "ASEAN": ["Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"],
    "European Union": ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"],
    "African Union": ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cabo Verde", "Cameroon", "Central African Republic", "Chad", "Comoros", "Congo", "Democratic Republic of the Congo", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Rwanda", "São Tomé and Príncipe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe"],
    "Middle East": ["Afghanistan", "Bahrain", "Cyprus", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman", "Palestine", "Qatar", "Saudi Arabia", "Syria", "Turkey", "United Arab Emirates", "Yemen"],
    "Latin America and Caribbean": ["Antigua and Barbuda", "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Chile", "Colombia", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Grenada", "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Suriname", "Trinidad and Tobago", "Uruguay", "Venezuela"]
}

# Reference countries for comparison
reference_countries = ["United States", "China"]

# Color palette mapping
pastel_palette = pc.qualitative.Pastel
color_map = {}

# Create a mapping of countries to pastel colors
country_list = [country for group in groups.values() for country in group]
for i, country in enumerate(country_list):
    color_map[country] = pastel_palette[i % len(pastel_palette)]

# Function to plot the graphs
for group_name, group_countries in groups.items():
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f"{group_name} vs. US", f"{group_name} vs. China"])

    for col, reference_country in enumerate(reference_countries, start=1):
        # Compute cosine similarity based on dual-use data
        cosine_sim_df = calculate_cosine_with_base_year(df_dual_use_processed, group_countries + reference_countries)
        filtered_df = cosine_sim_df.filter(like=f"Cosine Similarity {reference_country} vs", axis=1)

        # Rename columns to remove redundant text
        melted_df = cosine_sim_df[['New Year']].join(filtered_df)
        melted_df.columns = melted_df.columns.str.replace(f"Cosine Similarity {reference_country} vs ", "")

        # Convert to long format for Plotly
        melted_df = melted_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')
        melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

        # Calculate the average similarity for the group for each year
        group_df = melted_df[melted_df['Country'].isin(group_countries)]
        avg_group_similarity = group_df.groupby('New Year')['Similarity'].mean().reset_index()

        # Add lines for each country in the group
        for country in group_countries:
            country_data = melted_df[melted_df['Country'] == country]
            fig.add_trace(go.Scatter(
                x=country_data['New Year'],
                y=country_data['Similarity'],
                mode='lines+markers',
                name=country,
                line=dict(shape='spline', color=color_map.get(country, 'black')),
                showlegend=(col == 1),  # Show legend only for the first plot
                connectgaps=True
            ), row=1, col=col)

        # Add the average similarity line for the group
        fig.add_trace(go.Scatter(
            x=avg_group_similarity['New Year'],
            y=avg_group_similarity['Similarity'],
            mode='lines',
            name=f"Average {group_name}",
            line=dict(shape='spline', width=3, color='black'),
            showlegend=(col == 1)
        ), row=1, col=col)

    # Update layout
    fig.update_layout(
        title=f"Dual-Use Technology: Cosine Similarity Trends for {group_name} ({base_number})",
        xaxis_title="Year",
        yaxis_title="Cosine Similarity",
        yaxis=dict(range=[0, 1]),  # Applies to first subplot (US)
        yaxis2=dict(range=[0, 1], matches="y", title="Cosine Similarity"),  # Matches left side
        xaxis=dict(tickmode="array", tickvals=[year for year in range(1990, 2027, 2)]),
        xaxis2=dict(tickmode="array", tickvals=[year for year in range(1990, 2027, 2)]),
        showlegend=True,
        template="plotly_white",
        hovermode="closest")

    fig.show()


In [ ]:
import plotly.graph_objects as go
import plotly.colors as pc
from plotly.subplots import make_subplots

base_number = 2021

# Define groups for comparison
groups = {
    "ASEAN": ["Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"],
    "European Union": ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"],
    "African Union": ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cabo Verde", "Cameroon", "Central African Republic", "Chad", "Comoros", "Congo", "Democratic Republic of the Congo", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Rwanda", "São Tomé and Príncipe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe"],
    "Middle East": ["Afghanistan", "Bahrain", "Cyprus", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman", "Palestine", "Qatar", "Saudi Arabia", "Syria", "Turkey", "United Arab Emirates", "Yemen"],
    "Latin America and Caribbean": ["Antigua and Barbuda", "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Chile", "Colombia", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Grenada", "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Suriname", "Trinidad and Tobago", "Uruguay", "Venezuela"]
}

# Reference countries for comparison (excluding Germany and the UK)
reference_countries = ["United States", "China"]

# Color palette mapping
pastel_palette = pc.qualitative.Pastel
color_map = {}

# Create a mapping of countries to pastel colors
country_list = [country for group in groups.values() for country in group]
for i, country in enumerate(country_list):
    color_map[country] = pastel_palette[i % len(pastel_palette)]

# Function to plot the graphs
for group_name, group_countries in groups.items():
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f"{group_name} vs. US", f"{group_name} vs. China"])

    for col, reference_country in enumerate(reference_countries, start=1):
        # Filter the relevant cosine similarity columns
        cosine_sim_df = calculate_cosine_with_base_year(df_military_processed, group_countries + reference_countries)
        filtered_df = cosine_sim_df.filter(like=f"Cosine Similarity {reference_country} vs", axis=1)

        # Rename columns to remove redundant text
        melted_df = cosine_sim_df[['New Year']].join(filtered_df)
        melted_df.columns = melted_df.columns.str.replace(f"Cosine Similarity {reference_country} vs ", "")

        # Convert to long format for Plotly
        melted_df = melted_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')

        # Remove NaNs for proper rendering
        melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

        # Calculate the average similarity for the group for each year
        group_df = melted_df[melted_df['Country'].isin(group_countries)]
        avg_group_similarity = group_df.groupby('New Year')['Similarity'].mean().reset_index()

        # Add lines for each country in the group
        for country in group_countries:
            country_data = melted_df[melted_df['Country'] == country]
            fig.add_trace(go.Scatter(
                x=country_data['New Year'],
                y=country_data['Similarity'],
                mode='lines+markers',
                name=country,
                line=dict(shape='spline', color=color_map.get(country, 'black')),
                showlegend=(col == 1),  # Show legend only for the first plot
                connectgaps=True
            ), row=1, col=col)

        # Add the average similarity line for the group
        fig.add_trace(go.Scatter(
            x=avg_group_similarity['New Year'],
            y=avg_group_similarity['Similarity'],
            mode='lines',
            name=f"Average {group_name}",
            line=dict(shape='spline', width=3, color='black'),
            showlegend=(col == 1)
        ), row=1, col=col)

    # Update layout
        fig.update_layout(
            title=f"Military Technology: Cosine Similarity Trends for {group_name} ({base_number})",
            xaxis_title="Year",
            yaxis_title="Cosine Similarity",
            yaxis=dict(range=[0, 1]),  # Applies to first subplot (US)
            yaxis2=dict(range=[0, 1], matches="y", title="Cosine Similarity"),  # Matches left side
            xaxis=dict(tickmode="array", tickvals=[year for year in range(1990, 2027, 2)]),
            xaxis2=dict(tickmode="array", tickvals=[year for year in range(1990, 2027, 2)]),
            showlegend=True,
            template="plotly_white",
            hovermode="closest")


    fig.show()



In [ ]:
import plotly.graph_objects as go
import plotly.colors as pc
from plotly.subplots import make_subplots

base_number = 2021

# Define groups for comparison
groups = {
    "ASEAN": ["Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"],
    "European Union": ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"],
    "African Union": ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cabo Verde", "Cameroon", "Central African Republic", "Chad", "Comoros", "Congo", "Democratic Republic of the Congo", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Rwanda", "São Tomé and Príncipe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe"],
    "Middle East": ["Afghanistan", "Bahrain", "Cyprus", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman", "Palestine", "Qatar", "Saudi Arabia", "Syria", "Turkey", "United Arab Emirates", "Yemen"],
    "Latin America and Caribbean": ["Antigua and Barbuda", "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Chile", "Colombia", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Grenada", "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Suriname", "Trinidad and Tobago", "Uruguay", "Venezuela"]
}

# Reference countries for comparison (excluding Germany and the UK)
reference_countries = ["United Kingdom", "China"]

# Color palette mapping
pastel_palette = pc.qualitative.Pastel
color_map = {}

# Create a mapping of countries to pastel colors
country_list = [country for group in groups.values() for country in group]
for i, country in enumerate(country_list):
    color_map[country] = pastel_palette[i % len(pastel_palette)]

# Function to plot the graphs
for group_name, group_countries in groups.items():
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f"{group_name} vs. UK", f"{group_name} vs. China"])

    for col, reference_country in enumerate(reference_countries, start=1):
        # Filter the relevant cosine similarity columns
        cosine_sim_df = calculate_cosine_with_base_year(df_military_processed, group_countries + reference_countries)
        filtered_df = cosine_sim_df.filter(like=f"Cosine Similarity {reference_country} vs", axis=1)

        # Rename columns to remove redundant text
        melted_df = cosine_sim_df[['New Year']].join(filtered_df)
        melted_df.columns = melted_df.columns.str.replace(f"Cosine Similarity {reference_country} vs ", "")

        # Convert to long format for Plotly
        melted_df = melted_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')

        # Remove NaNs for proper rendering
        melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

        # Calculate the average similarity for the group for each year
        group_df = melted_df[melted_df['Country'].isin(group_countries)]
        avg_group_similarity = group_df.groupby('New Year')['Similarity'].mean().reset_index()

        # Add lines for each country in the group
        for country in group_countries:
            country_data = melted_df[melted_df['Country'] == country]
            fig.add_trace(go.Scatter(
                x=country_data['New Year'],
                y=country_data['Similarity'],
                mode='lines+markers',
                name=country,
                line=dict(shape='spline', color=color_map.get(country, 'black')),
                showlegend=(col == 1),  # Show legend only for the first plot
                connectgaps=True
            ), row=1, col=col)

        # Add the average similarity line for the group
        fig.add_trace(go.Scatter(
            x=avg_group_similarity['New Year'],
            y=avg_group_similarity['Similarity'],
            mode='lines',
            name=f"Average {group_name}",
            line=dict(shape='spline', width=3, color='black'),
            showlegend=(col == 1)
        ), row=1, col=col)

    # Update layout
        fig.update_layout(
            title=f"Military Technology: Cosine Similarity Trends for {group_name} ({base_number})",
            xaxis_title="Year",
            yaxis_title="Cosine Similarity",
            yaxis=dict(range=[0, 1]),  # Applies to first subplot (US)
            yaxis2=dict(range=[0, 1], matches="y", title="Cosine Similarity"),  # Matches left side
            xaxis=dict(tickmode="array", tickvals=[year for year in range(1990, 2027, 2)]),
            xaxis2=dict(tickmode="array", tickvals=[year for year in range(1990, 2027, 2)]),
            showlegend=True,
            template="plotly_white",
            hovermode="closest")


    fig.show()


## PCA Analysis 

In [ ]:
# Load GDP data, skipping first four rows to get correct column names
gdp_df = pd.read_csv("GDP.csv", skiprows=4)

gdp_df = gdp_df.drop(columns=['Unnamed: 68'])

# Convert wide format to long format
gdp_long = gdp_df.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
                        var_name="Year", value_name="GDP")

# Convert Year column to integer
gdp_long["Year"] = gdp_long["Year"].astype(int)

# Merge with sentences_df on country code and year
merged_df = sentence_df.merge(gdp_long, left_on=["Country", "Year"], right_on=["Country Code", "Year"], how="left")

# Display results
merged_df.head()

merged_df = merged_df.drop(columns=['Country Name_y', 'Country Code', "Indicator Name", 'Indicator Code'])

merged = merged_df.rename(columns={'Country Name_x': 'Country Name'})


In [ ]:
merged_df = merged_df.rename(columns={'Country Name_x': 'Country Name'})
merged_df

,Country,Session,Year,Sentence,Country Name,Topic,Embedding,group,GDP
0,BRB,73,2018,Let me begin by congratulating Ms. María Ferna...,Barbados,95,"[-0.036270916, -0.0026696608, 0.0026795529, -0...",Uncategorized,5.188858e+09
1,BRB,73,2018,"However, I would like to pause at this stage, ...",Barbados,-1,"[0.015536962, 0.029011553, 0.023046829, -0.011...",Uncategorized,5.188858e+09
2,BRB,73,2018,Those events include the transit of a tropical...,Barbados,-1,"[0.067203976, 0.04081179, 0.11460312, 0.070455...",Uncategorized,5.188858e+09
3,BRB,73,2018,Those events are of great concern because the ...,Barbados,-1,"[0.060707532, 0.074981414, 0.02815113, 0.03468...",Uncategorized,5.188858e+09
4,BRB,73,2018,"I ask myself, what does all that matter? Last ...",Barbados,1166,"[0.09178345, 0.048956443, 0.018212734, 0.02262...",Uncategorized,5.188858e+09
...,...,...,...,...,...,...,...,...,...
522324,HUN,69,2014,"We have to heed the advice of Nicholas Stern, ...",Hungary,1132,"[0.056570105, 0.06454281, 0.05719191, 0.002399...",Uncategorized,2.977526e+11
522325,HUN,69,2014,I have just cited the example of New York City,Hungary,946,"[0.12445413, -0.03829658, 0.05360141, 0.039285...",Uncategorized,2.977526e+11
522326,HUN,69,2014,A few hundred million dollars would have been ...,Hungary,-1,"[0.014341723, 0.11719479, 0.060230006, -0.0165...",Uncategorized,2.977526e+11
522327,HUN,69,2014,The New York example tells us that we are not ...,Hungary,-1,"[0.10041228, 0.006510497, 0.10341268, 0.040018...",Uncategorized,2.977526e+11


In [ ]:
# Filter DataFrames based on the group from the merged DataFrame
PCA_military = merged_df[merged_df["group"] == "Military Technology"]
PCA_dual_use = merged_df[merged_df["group"] == "Dual Use Technology"]
PCA_civilian = merged_df[merged_df["group"] == "Civilian Technology"]
PCA_capacity_building = merged_df[merged_df["group"] == "Capacity Building"]
PCA_grey_zone = merged_df[merged_df["group"] == "Grey Zone"]

# Saving the DataFrames with specific names for each group
PCA_military.to_csv('PCA_military.csv', index=False)
PCA_dual_use.to_csv('PCA_dual_use.csv', index=False)
PCA_civilian.to_csv('PCA_civilian.csv', index=False)
PCA_capacity_building.to_csv('PCA_capacity_building.csv', index=False)
PCA_grey_zone.to_csv('PCA_grey_zone.csv', index=False)


/Users/student/anaconda3/envs/Capstone/lib/python3.11/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning:

invalid value encountered in cast



In [ ]:
PCA_military 

,Country,Session,Year,Sentence,Country Name,Topic,Embedding,group,GDP
48,BRB,73,2018,"I repeat, it led to two world wars and an atom...",Barbados,0,"[-0.01074151, 0.08483641, -0.014866305, 0.0476...",Military Technology,5.188858e+09
374,SWE,73,2018,We are witnessing a deeply worrisome nuclear r...,Sweden,0,"[0.029459324, 0.041300286, -0.034867812, 0.035...",Military Technology,6.131991e+11
376,SWE,73,2018,The framework of the Treaty on the Non-Prolife...,Sweden,0,"[0.005601457, 0.04289692, -0.0617467, -0.04264...",Military Technology,6.131991e+11
1050,ALB,73,2018,We support all the actions taken by the intern...,Albania,2,"[-0.011582497, 0.044264536, 0.013668734, -0.06...",Military Technology,4.213154e+10
1210,BOL,73,2018,"The second major threat is the arms race, the ...","Bolivia, Plurinational State of",0,"[0.048457425, 0.045786474, -0.022934327, 0.061...",Military Technology,1.140141e+11
...,...,...,...,...,...,...,...,...,...
520349,TUR,69,2014,"If 2,000 people are killed by chemical weapons...",Türkiye,2,"[0.10385505, -0.017613554, -0.01746026, -0.005...",Military Technology,1.896628e+12
520352,TUR,69,2014,Let it be understood that the use of any kind...,Türkiye,2,"[0.080962926, 0.07141191, -0.031963583, -0.004...",Military Technology,1.896628e+12
521583,LUX,69,2014,The proliferation of weapons of mass destruct...,Luxembourg,0,"[-0.013056871, -0.0032054023, 0.026966093, 0.0...",Military Technology,7.201170e+10
521655,TKM,69,2014,We call for consistent strengthening of intern...,Turkmenistan,0,"[0.0061750985, 0.03824251, 0.01598714, -0.0742...",Military Technology,7.621077e+10


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:


# 1. Group the data by 5-year periods starting from 1994
PCA_military['Period'] = ((PCA_military['Year'] - 1994) // 5) * 5 + 1994  # Grouping by 5-year periods

# 2. Calculate the average GDP for each country in each period, excluding NaN values
gdp_df = PCA_military.groupby(['Country Name', 'Period'])['GDP'].apply(lambda x: x.dropna().mean()).reset_index()

# 3. Calculate the average embedding, excluding NaN values
embeddings_df = PCA_military.groupby(['Country Name', 'Period'])['Embedding'].apply(lambda x: np.mean(np.vstack(x[~x.isna()]), axis=0)).reset_index()

# 4. Merge embeddings and GDP data on Country and Period
merged_embeddings_gdp = pd.merge(embeddings_df, gdp_df, on=['Country Name', 'Period'])

# Drop any rows with NaN GDP after the merge
merged_embeddings_gdp = merged_embeddings_gdp.dropna(subset=['GDP'])

# 5. Grouped data by Country and Period for average embeddings and GDP
grouped_data = merged_embeddings_gdp.groupby(['Country Name', 'Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# 6. Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# 7. Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# 8. Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

sorted_periods = sorted(grouped_data['Period'].unique())  # Sort periods in ascending order

# 10. Plotting for each period (creating separate plots)
for period in sorted_periods:
    period_data = grouped_data[grouped_data['Period'] == period]
    
    # Prepare the plot for the current period
    fig = go.Figure()
    
    # Add scatter plots for each country with the size of the dot corresponding to average GDP
    for country in period_data['Country Name'].unique():
        country_data = period_data[period_data['Country Name'] == country]
        fig.add_trace(go.Scatter(
            x=country_data['PCA1'],
            y=country_data['PCA2'],
            mode='markers',
            name=country,
            marker=dict(
                size=country_data['avg_gdp'] / 100000000000,  # Scale size for visualization
                opacity=0.6,
                line=dict(width=1, color='black')
            ),
            #visible='legendonly'  # Deselect all by default
        ))

    # Update layout with labels and title for each period
    fig.update_layout(
        title=f"PCA of Embeddings for Period {period} with GDP Size",
        xaxis_title='PCA 1',
        yaxis_title='PCA 2',
        showlegend=True,
        hovermode='closest'
    )

    # Show the plot for the current period
    fig.show()


NameError: name 'PCA_military' is not defined

In [ ]:
from sklearn.preprocessing import MinMaxScaler



In [ ]:


# 1. Group the data by 5-year periods starting from 1994
PCA_military['Period'] = ((PCA_military['Year'] - 1994) // 5) * 5 + 1994  # Grouping by 5-year periods

# 2. Calculate the average GDP for each country in each period, excluding NaN values
gdp_df = PCA_military.groupby(['Country Name', 'Period'])['GDP'].apply(lambda x: x.dropna().mean()).reset_index()

scaler = MinMaxScaler()

# Scale the GDP values and add to a new column 'scaled_gdp'
gdp_df['scaled_gdp'] = scaler.fit_transform(gdp_df[['GDP']])


# 3. Calculate the average embedding, excluding NaN values
embeddings_df = PCA_military.groupby(['Country Name', 'Period'])['Embedding'].apply(lambda x: np.mean(np.vstack(x[~x.isna()]), axis=0)).reset_index()

# 4. Merge embeddings and GDP data on Country and Period
merged_embeddings_gdp = pd.merge(embeddings_df, gdp_df, on=['Country Name', 'Period'])

# Drop any rows with NaN GDP after the merge
merged_embeddings_gdp = merged_embeddings_gdp.dropna(subset=['GDP'])

# 5. Grouped data by Country and Period for average embeddings and GDP
grouped_data = merged_embeddings_gdp.groupby(['Country Name', 'Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# 6. Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# 7. Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# 8. Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

# 9. Perform K-Means clustering on the PCA results
kmeans = KMeans(n_clusters=4, random_state=42)  # You can adjust the number of clusters
grouped_data['Cluster'] = kmeans.fit_predict(grouped_data[['PCA1', 'PCA2']])

# 10. Sort periods from oldest to newest
sorted_periods = sorted(grouped_data['Period'].unique())  # Sort periods in ascending order

# 11. Plotting for each period (creating separate plots)
for period in sorted_periods:
    period_data = grouped_data[grouped_data['Period'] == period]
    
    # Prepare the plot for the current period
    fig = go.Figure()
    
    # Add scatter plots for each country with the size of the dot corresponding to average GDP
    fig.add_trace(go.Scatter(
        x=period_data['PCA1'],
        y=period_data['PCA2'],
        mode='markers',
        name=f'Period {period}',
        marker=dict(
            size=period_data['avg_gdp'] / 100000000000,  # Scale size for visualization
            opacity=0.6,
            line=dict(width=1, color='black'),
            color=period_data['Cluster'],  # Color points by cluster
            colorscale='Viridis',  # Color scale for the clusters
            colorbar=dict(title='Cluster')  # Color bar for clusters
        ),
        text=period_data['Country Name'],  # Show country name on hover
    ))

    # Update layout with labels and title for each period
    fig.update_layout(
        title=f"PCA of Embeddings for Period {period} with GDP Size and K-Means Clusters",
        xaxis_title='PCA 1',
        yaxis_title='PCA 2',
        showlegend=True,
        hovermode='closest'
    )

    # Show the plot for the current period
    fig.show()


In [ ]:
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Group the data by 5-year periods starting from 1994
PCA_military['Period'] = ((PCA_military['Year'] - 1994) // 5) * 5 + 1994  # Grouping by 5-year periods

# Calculate the average GDP for each country in each period, excluding NaN values
gdp_df = PCA_military.groupby(['Country Name', 'Period'])['GDP'].apply(lambda x: x.dropna().mean()).reset_index()

# Calculate the average embedding, excluding NaN values
embeddings_df = PCA_military.groupby(['Country Name', 'Period'])['Embedding'].apply(lambda x: np.mean(np.vstack(x[~x.isna()]), axis=0)).reset_index()

# Merge embeddings and GDP data on Country and Period
merged_embeddings_gdp = pd.merge(embeddings_df, gdp_df, on=['Country Name', 'Period'])

# Drop any rows with NaN GDP after the merge
merged_embeddings_gdp = merged_embeddings_gdp.dropna(subset=['GDP'])

# Grouped data by Country and Period for average embeddings and GDP
grouped_data = merged_embeddings_gdp.groupby(['Country Name', 'Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

# Perform DBSCAN clustering on the PCA results
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Parameters can be adjusted
grouped_data['Cluster'] = dbscan.fit_predict(grouped_data[['PCA1', 'PCA2']])

# Sort periods from oldest to newest
sorted_periods = sorted(grouped_data['Period'].unique())  # Sort periods in ascending order

# Plotting for each period (creating separate plots)
for period in sorted_periods:
    period_data = grouped_data[grouped_data['Period'] == period]
    
    # Prepare the plot for the current period
    fig = go.Figure()
    
    # Add scatter plots for each country with the size of the dot corresponding to average GDP
    fig.add_trace(go.Scatter(
        x=period_data['PCA1'],
        y=period_data['PCA2'],
        mode='markers',
        name=f'Period {period}',
        marker=dict(
            size=period_data['avg_gdp'] / 100000000000,  # Scale size for visualization
            opacity=0.6,
            line=dict(width=1, color='black'),
            color=period_data['Cluster'],  # Color points by cluster
            colorscale='Viridis',  # Color scale for the clusters
            colorbar=dict(title='Cluster')  # Color bar for clusters
        ),
        text=period_data['Country Name'],  # Show country name on hover
    ))

    # Update layout with labels and title for each period
    fig.update_layout(
        title=f"PCA of Embeddings for Period {period} with GDP Size and DBSCAN Clusters",
        xaxis_title='PCA 1',
        yaxis_title='PCA 2',
        showlegend=True,
        hovermode='closest'
    )

    # Show the plot for the current period
    fig.show()


In [ ]:
# Display the rows where data is missing in any column
missing_data_rows = merged_embeddings_gdp[merged_embeddings_gdp.isnull().any(axis=1)]
missing_data_rows


,Country Name,Period,Embedding,GDP
0,Afghanistan,1994,"[0.036915474, 0.070349604, 0.031502873, 0.0217...",NaN
184,Cuba,1994,"[-0.055764496, 0.037322585, 0.04110789, 0.0002...",NaN
185,Cuba,1999,"[0.025159057, 0.04103466, 0.02013264, -0.00956...",NaN
186,Cuba,2004,"[0.014553455, 0.07134506, 0.013625699, 0.00568...",NaN
187,Cuba,2009,"[-0.011933499, 0.070004076, -0.0075313207, 0.0...",NaN
188,Cuba,2014,"[-0.038152855, 0.102020584, -0.023920141, 0.00...",NaN
189,Cuba,2019,"[-0.0021252837, 0.029470934, -0.025233084, 0.0...",NaN
211,Djibouti,1994,"[-0.042144902, 0.048103444, -0.009869823, -0.0...",NaN
319,Holy See (Vatican City State),2004,"[0.0016307281, 0.08461153, 0.00057803304, 0.00...",NaN
320,Holy See (Vatican City State),2009,"[-0.015786372, 0.09805957, -0.028198559, 0.010...",NaN


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Drop rows with NaN values in the GDP column
PCA_military_clean = PCA_military.dropna(subset=['GDP'])

# Create a new dataframe for the average embeddings and GDP over the 5-year periods
PCA_military_clean['Year Period'] = ((PCA_military_clean['Year'] - 1994) // 5) * 5 + 1994

# Group by country and year period to get the average embedding for each group
grouped_data = PCA_military_clean.groupby(['Country Name', 'Year Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

# Prepare the plot
fig = go.Figure()

# Add scatter plots for each country with the size of the dot corresponding to average GDP
for country in grouped_data['Country Name'].unique():
    country_data = grouped_data[grouped_data['Country Name'] == country]
    fig.add_trace(go.Scatter(
        x=country_data['PCA1'],
        y=country_data['PCA2'],
        mode='markers',
        name=country,
        marker=dict(
            size=country_data['avg_gdp'] / 1000,  # Scale size for visualization
            opacity=0.6,
            line=dict(width=1, color='black')
        ),
        visible='legendonly'  # Deselect all by default
    ))

# Update layout with labels and title
fig.update_layout(
    title="PCA of Embeddings by Country with GDP Size",
    xaxis_title='PCA 1',
    yaxis_title='PCA 2',
    showlegend=True,
    hovermode='closest'
)

# Show the plot
fig.show()


In [ ]:
# 4. Perform PCA on the embeddings
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(merged_embeddings_gdp['Embedding']))

pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_embeddings)

# Add PCA results to the dataframe
merged_embeddings_gdp['PCA1'] = pca_result[:, 0]
merged_embeddings_gdp['PCA2'] = pca_result[:, 1]

# 5. Create the PCA plot using Plotly
fig = go.Figure()

# Add traces for each country
for country in merged_embeddings_gdp['Country Name'].unique():
    country_data = merged_embeddings_gdp[merged_embeddings_gdp['Country Name'] == country]
    fig.add_trace(go.Scatter(
        x=country_data['PCA1'],
        y=country_data['PCA2'],
        mode='markers',
        name=country,
        marker=dict(
            size=country_data['GDP'] / 1000,  # Scale GDP for marker size
            opacity=0.7
        ),
        visible='legendonly'  # Hide the traces by default, can be toggled via legend
    ))

# 6. Update layout
fig.update_layout(
    title='PCA of Military Technology Embeddings (Grouped by 5-Year Periods)',
    xaxis_title='PCA Component 1',
    yaxis_title='PCA Component 2',
    showlegend=True,
    hovermode='closest',
    legend=dict(
        title="Country",
        x=1.05,  # Place legend outside the plot area
        y=1
    )
)

# Show the plot
fig.show()